In [ ]:
import pandas as pd
import math
infile1 = r'./excels/sortedAll2004-2021.csv'
infile2 = r'./excels/sortedAll2004-2021Remaining.csv'
rowCount1 = 1048690
rowCount2 = 176527

# df = pd.read_csv("train.csv", dtype={"YearMade": "int16"})
# data = pd.read_csv(infile1, skiprows = 0, nrows=rowCount1, dtype={"Price":"float32", "Qty":"int16", "Owned":"int16", "Value": "float32"})
data = pd.read_csv(infile1, skiprows = 0, nrows=rowCount1)
data = data.drop(['Unnamed: 0', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14'], axis=1)
data2 = pd.read_csv(infile2, skiprows = 0, nrows=rowCount2)
data2 = data2.drop(['Unnamed: 0'], axis=1)

for index, row in data.iterrows():
    if row['deltOwned'][0]=='N':
        data.loc[index, "deltOwned"] = 0
        data.loc[index, "Value"] = float(row['Price'])*int(row['Qty'])

for index, row in data2.iterrows():
    if row['deltOwned'][0]=='N':
        data2.loc[index, "deltOwned"] = 0
        data2.loc[index, "Value"] = float(row['Price'])*int(row['Qty'])

data = data.append(data2, ignore_index = True)

data

/Users/patrick/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# data.loc[data['Insider Name'] == 'Neuson Kramer Baumaschinen AG'].loc[data['Price'] == 23000000.00]
data=data.drop(data.loc[data['Insider Name'] == 'Neuson Kramer Baumaschinen AG'].loc[data['Price'] == 23000000.00].index.item())
data=data.reset_index(drop=True)
# data

# data.loc[131594]

### gameplan:
-iterate through rows
-if insider makes a purchase, add them to dictionary of traders---initialize amount to purchased. 
-dictionary has format key=name (as seen in dataframe) value=dictionary which includes: queue of trades (tuple with ticker, price, trade date, quantity of shares, total value). updates dynamically with time. if sale is made and all shares of a given previous purchase is sold, then it is popped from the queue and the next purchase up is decremented accordingly. 

-also sum the amount spent in purchases for each insider to get their return

In [3]:
from datetime import datetime

# insiders = {
#     "ExampleName":{"Title(s)":[], "Affiliated Company":"Comp Name", "Profit":0, "Amount Inv":0, "Return":0, "Equity Dist":{}, "Trade Count":0, "Shares Traded":0, "Shares Held":0, "Trade Data":[], "Trades":[], "Trades by Ticker":{'TickerExample':[[]]}} #could also have itemized equity holdings by each of their company holdings, etc.
# }
insiders = {}

for index, row in data.iterrows():
    
    if float(row["Price"]) == 0:
        continue  #may be a problem when they sell high-value yet 0-price purchases...
    
    if row['Insider Name'] not in insiders and row['Trade Type']=="P - Purchase":
        insiders[row['Insider Name']] = { "Title(s)":list(row['Title'].split(', ')), 
                                         "Affiliated Company":row["Company Name"], 
                                         "Profit":0, 
                                         "Amount Inv":float(row["Value"]),
                                         "Return":0,
                                         "Equity Dist": { row["Ticker"] : float(row["Value"])}, 
                                         "Trade Count":1, 
                                         "Shares Traded":int(row["Qty"]), 
                                         "Shares Held":int(row["Qty"]), 
                                         "Trade Data":[row.tolist()], 
                                         "Trades by Ticker":{row["Ticker"] : [ [ int(row["Qty"]), float(row["Price"]), row['Trade Date'] ] ]},
                                         "Avg Hold Stats":[0,0, 0] #---(qty*daysHeld, totalQty, currentAvg). get difference between two dates. later we can do average holding time by ticker
                                        }
    elif row['Trade Type']=="P - Purchase":
        
        insiders[row['Insider Name']]["Amount Inv"] += float(row["Value"])        
        
        if row["Ticker"] in insiders[row['Insider Name']]["Equity Dist"]:
            insiders[row['Insider Name']]["Equity Dist"][row["Ticker"]]+=float(row["Value"])
        else:
            insiders[row['Insider Name']]["Equity Dist"][row["Ticker"]] = float(row["Value"])
        
        insiders[row['Insider Name']]["Trade Count"]+=1
        insiders[row['Insider Name']]["Shares Traded"] += int(row["Qty"])
        insiders[row['Insider Name']]["Shares Held"] += int(row["Qty"])
        insiders[row['Insider Name']]["Trade Data"].append(row.tolist())
        
#         insiders[row['Insider Name']]["Trades"].append([ row["Ticker"], int(row["Qty"].replace(',', '')), float(row["Price"].replace(',', '')), row['Trade Date'] ])
        if row["Ticker"] in insiders[row['Insider Name']]["Trades by Ticker"]:
            insiders[row['Insider Name']]["Trades by Ticker"][row['Ticker']].append([ int(row["Qty"]), float(row["Price"]), row['Trade Date'] ])
        else:
            insiders[row['Insider Name']]["Trades by Ticker"][row['Ticker']] = [[ int(row["Qty"]), float(row["Price"]), row['Trade Date'] ]]
    
    elif row['Trade Type'][0]=='S' and row['Insider Name'] in insiders and (row['Ticker'] in insiders[row['Insider Name']]['Equity Dist'] ):    
        #what if sale *qty* is greater than total equity---in that case just subtract the total held qty down to 0 and move on
        qty = int(row["Qty"])*(-1)
        sellPrice = float(row["Price"])
        while qty > 0 and insiders[row['Insider Name']]["Trades by Ticker"][row['Ticker']]:
            heldQtyTop = insiders[row['Insider Name']]["Trades by Ticker"][row['Ticker']][0][0] #dynamic qty of purchased shares
            buyPrice = insiders[row['Insider Name']]["Trades by Ticker"][row['Ticker']][0][1]
            
            if qty > heldQtyTop:
                insiders[row['Insider Name']]["Profit"]+=(sellPrice*heldQtyTop - buyPrice*heldQtyTop)
                qty -= heldQtyTop
                
                start = datetime.strptime(insiders[row['Insider Name']]["Trades by Ticker"][row['Ticker']][0][2], "%m/%d/%Y")
                end =   datetime.strptime(row['Trade Date'], "%m/%d/%Y")
                timeHeld = (end.date() - start.date()).days
                insiders[row['Insider Name']]['Avg Hold Stats'][0] += (heldQtyTop*timeHeld)
                insiders[row['Insider Name']]['Avg Hold Stats'][1] += heldQtyTop
#                 insiders[row['Insider Name']]['Avg Hold Stats'][2] = insiders[row['Insider Name']]['Avg Hold Stats'][0]/insiders[row['Insider Name']]['Avg Hold Stats'][1]
                
                insiders[row['Insider Name']]["Trades by Ticker"][row['Ticker']].pop(0)
                
            else:
                #simply decrement from the current holding qty and change the profit accordingly
                insiders[row['Insider Name']]["Profit"]+=(sellPrice*qty - buyPrice*qty)
                insiders[row['Insider Name']]["Trades by Ticker"][row['Ticker']][0][0] -= qty
                qty = 0
                
                start = datetime.strptime(insiders[row['Insider Name']]["Trades by Ticker"][row['Ticker']][0][2], "%m/%d/%Y")
                end =   datetime.strptime(row['Trade Date'], "%m/%d/%Y")
                timeHeld = (end.date() - start.date()).days
                insiders[row['Insider Name']]['Avg Hold Stats'][0] += (qty*timeHeld)
                insiders[row['Insider Name']]['Avg Hold Stats'][1] += qty
#                 insiders[row['Insider Name']]['Avg Hold Stats'][2] = insiders[row['Insider Name']]['Avg Hold Stats'][0]/insiders[row['Insider Name']]['Avg Hold Stats'][1]
                
#         print(insiders[row['Insider Name']]["Amount Inv"], row['Insider Name'], insiders[row['Insider Name']], '-----\n', row, "\n\n")
        if insiders[row['Insider Name']]["Profit"] != 0:
            insiders[row['Insider Name']]["Return"]=round(insiders[row['Insider Name']]["Profit"]/insiders[row['Insider Name']]["Amount Inv"], 2)
        
insiders

ValueError: time data '1/2/04' does not match format '%m/%d/%Y'

In [7]:
for insider in insiders:
#     print(insider, insiders[insider], "\n")
    if insiders[insider]['Avg Hold Stats'][1] != 0:
        insiders[insider]['Avg Hold Stats'][2]=insiders[insider]['Avg Hold Stats'][0]/insiders[insider]['Avg Hold Stats'][1]

##### TODO: sum up the amount spent on purchases by each insider so that we can get the return for each individual
##### sum up the amount spent by each position and the return of each position. see which has highest
##### for (position, individual) get total profit, average and median return---profit/spent


##### eventually, want to get the current value (or maybe average of the past year) of presently-held stocks so that returns can be measured for insiders who bought and held until the present

##### have to also have a way of measuring returns per year of investing...eg. do they get a high return in a short period of time?
##### ^to discover this, consider taking the range of time between buying and selling a given stock. take an average holding period for each stock, for example. IOW, are they making quick money on certain stocks?
##### how do we combine our view of time a stock is held with the return of the holder? maybe a 2-d scatterplot. (add in total volume $ traded for a third dimension)


In [8]:
# returnsList = []
# for insider in insiders:
# #     if insiders[insider]['Profit']>0:
#     returnsList.append([ insiders[insider]['Return'], insiders[insider]['Trade Count'], insider, insiders[insider]['Amount Inv'], insiders[insider]['Title(s)'] ])

# returnsList = sorted(returnsList, 
#        key=lambda x: x[0], reverse=True)
# returnsList

In [46]:
returnsList = []
exceptCount = 0
for insider in insiders:
    # this len/continue part is to filter by the number of trades made. we may not want to follow 1-hit wonders (or 2- / 3-hit wonders)
    if len(insiders[insider]['Trade Data'])<4:
        continue   #we don't want people who do one big trade to skew results
    
#     if insiders[insider]['Profit']>0:
    try:
        returnsList.append([ insiders[insider]['Return']/insiders[insider]['Avg Hold Stats'][2], insiders[insider]['Return'], insiders[insider]['Trade Count'], insider, insiders[insider]['Amount Inv'], insiders[insider]['Title(s)'] ])
    except:
        exceptCount+=1
        returnsList.append([ 0, insiders[insider]['Return'], insiders[insider]['Trade Count'], insider, insiders[insider]['Amount Inv'], insiders[insider]['Title(s)'] ])

returnsList = sorted(returnsList, 
       key=lambda x: x[0], reverse=True)
# print(exceptCount)
# try to understand the 
returnsList

[[0.9150292653349117,
  34.8,
  13,
  'Bass Ronald Scott',
  87527.0,
  ['Principal Accounting Officer']],
 [0.7945348551456735,
  2.39,
  18,
  'Crabtree Michael L',
  188860.0,
  ['Pres of Pharmacy Division']],
 [0.64, 0.64, 4, 'Ellis James L', 5536.0, ['Dir']],
 [0.57625, 13.83, 5, 'Knight Bryce', 256000.0, ['CEO', '10%']],
 [0.48949999999999994,
  9.79,
  4,
  'Milder Robert',
  161635.0,
  ['Chief Operations Officer']],
 [0.46350758853288365, 30.54, 6, 'Berg Ian', 37230.0, ['Dir']],
 [0.39714285714285713,
  2.78,
  4,
  'Anthony Michael F',
  711251.0,
  ['COB,CEO', 'Pres']],
 [0.3352388357855433, 33.1, 7, 'Toh Kok Soon', 1766401.0, ['CEO', '10%']],
 [0.29, 0.29, 4, 'Hyperion Trading Segregated Portfolio', 10754397.0, ['10%']],
 [0.22733957219251338, 170.05, 5, 'Rothermel Robert B', 128079.0, ['Dir']],
 [0.21666666666666667, 1.3, 6, 'Dissinger Debra E', 7900.0, ['EVP']],
 [0.15, 0.15, 4, 'Allen Robert B', 362012.0, ['Pres', 'CEO']],
 [0.1486587698978199, 24.4, 20, 'Garneau Edouard

In [47]:
#TODO: sum up the amount spent on purchases by each insider so that we can get the return for each individual
#sum up the amount spent by each position and the return of each position. see which has highest
#for (position, individual) get total profit, average and median return---profit/spent


#eventually, want to get the current value (or maybe average of the past year) of presently-held stocks so that returns can be measured for insiders who bought and held until the present

#have to also have a way of measuring returns per year of investing...eg. do they get a high return in a short period of time?
#^to discover this, consider taking the range of time between buying and selling a given stock. take an average holding period for each stock, for example. IOW, are they making quick money on certain stocks?
#how do we combine our view of time a stock is held with the return of the holder? maybe a 2-d scatterplot. (add in total volume $ traded for a third dimension)

#DO WEIGHTED RETURNS!!! BY THE AMOUNT SPENT???


In [48]:
returnsList

[[0.9150292653349117,
  34.8,
  13,
  'Bass Ronald Scott',
  87527.0,
  ['Principal Accounting Officer']],
 [0.7945348551456735,
  2.39,
  18,
  'Crabtree Michael L',
  188860.0,
  ['Pres of Pharmacy Division']],
 [0.64, 0.64, 4, 'Ellis James L', 5536.0, ['Dir']],
 [0.57625, 13.83, 5, 'Knight Bryce', 256000.0, ['CEO', '10%']],
 [0.48949999999999994,
  9.79,
  4,
  'Milder Robert',
  161635.0,
  ['Chief Operations Officer']],
 [0.46350758853288365, 30.54, 6, 'Berg Ian', 37230.0, ['Dir']],
 [0.39714285714285713,
  2.78,
  4,
  'Anthony Michael F',
  711251.0,
  ['COB,CEO', 'Pres']],
 [0.3352388357855433, 33.1, 7, 'Toh Kok Soon', 1766401.0, ['CEO', '10%']],
 [0.29, 0.29, 4, 'Hyperion Trading Segregated Portfolio', 10754397.0, ['10%']],
 [0.22733957219251338, 170.05, 5, 'Rothermel Robert B', 128079.0, ['Dir']],
 [0.21666666666666667, 1.3, 6, 'Dissinger Debra E', 7900.0, ['EVP']],
 [0.15, 0.15, 4, 'Allen Robert B', 362012.0, ['Pres', 'CEO']],
 [0.1486587698978199, 24.4, 20, 'Garneau Edouard

In [49]:
import collections
##commented here are with multiple attributes for each position, whereas subsequent uncommented is dictionary of only returns
# positionStats = {}
# for row in returnsList:
#     for position in row[4]:
#         if row[0] !=0:
#             if position not in positionStats:
#                 positionStats[position]={}
#                 positionStats[position]['Returns'] = []

#             positionStats[position]['Returns'].append(row[0])
# positionStats
positionStats = {}
for row in returnsList:
    print(row)
    
#     for position in row[4]:  # for before the time ratio was done at first index and row/list length was shorter
    for position in row[5]:        
        
        if row[0] !=0:
            if position not in positionStats:
                positionStats[position]=[]

            positionStats[position].append(row[0])

# d = {"one": [(1,3),(1,4)], "two": [(1,2),(1,2),(1,3)], "three": [(1,1)]}
# >>> for k in sorted(d, key=lambda k: len(d[k]), reverse=True):
#         print k,


# two one three

#to order the previously-made dictionary or returns by position, we have this next line
positionStats = collections.OrderedDict(sorted(positionStats.items(), key= lambda x: len(x[1]), reverse=True))

positionStats

[0.9150292653349117, 34.8, 13, 'Bass Ronald Scott', 87527.0, ['Principal Accounting Officer']]
[0.7945348551456735, 2.39, 18, 'Crabtree Michael L', 188860.0, ['Pres of Pharmacy Division']]
[0.64, 0.64, 4, 'Ellis James L', 5536.0, ['Dir']]
[0.57625, 13.83, 5, 'Knight Bryce', 256000.0, ['CEO', '10%']]
[0.48949999999999994, 9.79, 4, 'Milder Robert', 161635.0, ['Chief Operations Officer']]
[0.46350758853288365, 30.54, 6, 'Berg Ian', 37230.0, ['Dir']]
[0.39714285714285713, 2.78, 4, 'Anthony Michael F', 711251.0, ['COB,CEO', 'Pres']]
[0.3352388357855433, 33.1, 7, 'Toh Kok Soon', 1766401.0, ['CEO', '10%']]
[0.29, 0.29, 4, 'Hyperion Trading Segregated Portfolio', 10754397.0, ['10%']]
[0.22733957219251338, 170.05, 5, 'Rothermel Robert B', 128079.0, ['Dir']]
[0.21666666666666667, 1.3, 6, 'Dissinger Debra E', 7900.0, ['EVP']]
[0.15, 0.15, 4, 'Allen Robert B', 362012.0, ['Pres', 'CEO']]
[0.1486587698978199, 24.4, 20, 'Garneau Edouard A', 54970158.0, ['CEO']]
[0.14296437869822484, 1.39, 4, 'Whitney

[0.0005323202751057204, 0.29, 5, 'Brova Jacquelin J', 191493.0, ['VP HR']]
[0.0005316755024981884, 0.41, 24, 'Pinola Richard J', 900754.0, ['Dir']]
[0.0005314717503953623, 0.83, 8, 'Alexander Andrew M', 421813.0, ['Pres']]
[0.0005292635116213946, 0.45, 4, 'Nunneley Mark', 515520.0, ['Chief Accounting Officer']]
[0.0005286311313146125, 1.03, 4, 'Bonner E V Jr', 886250.0, ['EVP', 'Secretary', 'Gen. Counsel']]
[0.0005284025028860995, 1.0, 6, 'Andrews Kirkland B', 1482765.0, ['EVP', 'CFO']]
[0.0005277752522956632, 0.86, 5, 'Earley Michael', 88689.0, ['Pres', 'CEO']]
[0.0005274291486291906, 1.47, 31, 'Hileman Donald P.', 175103.0, ['EVP', 'CFO']]
[0.000527194734601657, 0.23, 12, 'Bridgford Richard Eugene', 28770.0, ['VP', 'Ind. Relations & Safety']]
[0.0005270472081639426, 0.33, 4, 'Corasanti Joseph J', 479297.0, ['Dir']]
[0.0005270202442697005, 0.3, 10, 'McCormick Paul', 162101.0, ['Dir']]
[0.000527016901173924, 0.2, 5, 'Curtis Mark D', 13293.0, ['SVP', 'Treasurer']]
[0.0005267291526927823

[2.5210084033613446e-05, 0.06, 5, 'Glenn T Michael', 964146.0, ['Dir']]
[2.503152348968152e-05, 0.01, 6, 'Tama Carol J', 102841.0, ['Dir']]
[2.4643450665559166e-05, 0.07, 24, 'Hackemer Dean', 370702.0, ['EVP (sub)']]
[2.4633477690810125e-05, 0.04, 10, 'Gorelick Jamie S', 1413960.0, ['Dir']]
[2.444898305010001e-05, 0.01, 4, 'Avramovich Daniel W', 1288390.0, ['CEO']]
[2.4410089503661513e-05, 0.05, 5, 'McDonnell John J Jr', 1127965.0, ['COB', 'CEO']]
[2.4330641284343105e-05, 0.01, 57, 'Leucadia National Corp', 1393139870.0, ['10%']]
[2.41954256489722e-05, 0.04, 23, 'Goepel Patrick', 1715999.0, ['CEO']]
[2.4108523499367418e-05, 0.07, 6, 'Cherbak Anthony', 255326.0, ['COO']]
[2.3842104099466162e-05, 0.03, 9, 'Havener Arthur L Jr', 512794.48, ['Dir']]
[2.3731581212437357e-05, 0.02, 20, 'Stewart John D', 473911.0, ['Dir']]
[2.3446658851113717e-05, 0.02, 8, 'Walker John H', 400087.0, ['Dir']]
[2.342509970255959e-05, 0.05, 5, 'Tomczyk Fredric J', 1748101.0, ['Dir']]
[2.3325126344434366e-05, 0.0

[0, 0, 15, 'Korthals Robert W', 193638.96, ['Dir']]
[0, 0, 5, 'Glace Janet', 174460.0, ['10%']]
[0, 0, 10, 'Glace Robert W', 545093.0, ['10%']]
[0, 0, 7, 'Dale Bruce', 201368.0, ['Pres', 'CEO']]
[0, 0, 6, 'Hanna James V', 38638.0, ['Dir']]
[0, 0, 11, 'Coccia Joseph A', 368828.0, ['Dir']]
[0, 0, 4, 'Carlander Carrie', 49900.0, ['CFO', 'Treasurer', 'VP Finance']]
[0, 0, 10, 'Prince Charles', 1680494.0, ['CEO']]
[0, 0, 13, 'McMurray Louis C', 464070.0, ['Dir', '10%']]
[0, 0, 4, 'Tollett Leland', 606700.0, ['Dir']]
[0, 0, 9, 'Robo James L', 5377087.68, ['Dir']]
[0, 0, 7, 'York James Edward', 3192860.0, ['Dir']]
[0, 0, 7, 'Hunt Johnnie B', 13940120.0, ['10%']]
[0, 0, 5, 'Vaczy Catherine M', 235172.96, ['EVP', 'GC']]
[0, 0, 6, 'Wilson Karen M', 264358.0, ['Dir']]
[0, 0, 6, 'Lerch Stephen E', 79092.0, ['CFO', 'COO']]
[0, 0, 9, 'Ruppert Craig A', 803542.0, ['Dir']]
[0, 0, 7, 'Abel Douglas', 38041.0, ['Pres', 'CEO']]
[0, 0, 7, 'Schechter Robert', 1720739.0, ['COB', 'CEO']]
[0, 0, 9, 'Erwin Samu

[0, 0, 4, 'Porter Biggs C', 213980.0, ['CFO']]
[0, 0.0, 12, 'Wasserman Jack Gumpert', 260342.0, ['Dir']]
[0, 0, 4, 'Deboer Allan', 80187.0, ['Dir']]
[0, 0, 17, 'Robins Patrick S', 184304.0, ['Dir']]
[0, 0, 36, "O'Keeffe Charles B", 592001.0, ['Dir']]
[0, 0, 5, 'Newman Stephen L', 297578.0, ['COO']]
[0, 0.55, 22, 'Kramer Lawrence S', 756127.5, ['Dir']]
[0, 0, 5, 'Johnson Carlton M', 54419.770000000004, ['Dir']]
[0, 0, 8, 'Kirincic Paul E', 234384.0, ['Dir']]
[0, 0, 4, 'Fiske McNeil S. Jr', 359393.0, ['Pres', 'CEO']]
[0, 0, 11, 'Majteles Robert J', 1284754.0, ['Dir']]
[0, -0.01, 10, 'Schramm David', 370323.0, ['CEO', 'Pres']]
[0, 0, 5, 'White Augustus A', 49278.0, ['Dir']]
[0, 0, 7, 'Gross Stormie L', 3356.0, ['VPRisk Mgmt', 'Compliance Office']]
[0, 0, 5, 'Nicholas David', 67432.8, ['Dir']]
[0, 0, 5, 'Wagers Thomas B Sr', 39078.0, ['COO of Subsidiary']]
[0, 0, 7, 'Helton Roy Tom', 160255.0, ['CHRO']]
[0, 0, 7, 'Kling William H', 77148.0, ['Dir']]
[0, 0, 6, 'McCain Thomas G', 38339.0, ['

[0, 0, 4, 'Cherry James C.', 432631.0, ['CEO']]
[0, 0.3, 5, 'Prochaska Joseph J Jr', 212152.0, ['Dir']]
[0, 0, 32, 'Abel Robert A', 296026.0, ['Dir']]
[0, 0, 7, 'Beatty David R', 1160540.0, ['Dir']]
[0, 0, 4, 'Murray Robin E', 7646.0, ['SVP']]
[0, 0, 6, 'Stokely John D Jr', 116243.0, ['Dir']]
[0, 0, 4, 'Quest Diagnostics Inc', 61451720.0, ['10%']]
[0, 0.19, 5, 'Dummer Arthur Oleen', 319939.0, ['Dir']]
[0, 0, 20, 'Schrecongost James R', 91721.0, ['Dir']]
[0, 0, 5, 'Connolly Michael I', 51013.909999999996, ['Co-CEO']]
[0, 0, 4, 'Towell Joseph Hampton', 573957.0, ['Pres', 'CEO']]
[0, 0, 9, 'Van Oort Douglas M', 28837.0, ['COB', 'CEO']]
[0, 0, 4, 'Demarcus William M', 44145.0, ['Chief Banking Officer']]
[0, 0, 32, 'Shott Michael R', 743653.0, ['Dir']]
[0, 0, 9, 'Deer Jill V', 81075.0, ['Dir']]
[0, 0, 6, 'Moskovitz Ronald Bruce', 140610.0, ['Dir']]
[0, 0, 7, 'Dewet Pierre J', 528128.0, ['Dir']]
[0, 0, 13, 'Georgescu Peter A', 1232536.0, ['Dir']]
[0, 0, 5, 'La Forgia Robert M', 386450.0, ['D

[0, 0, 5, 'Wilson Hugh Steven', 387230.0, ['Dir']]
[0, 0, 6, 'Tuxis Corp', 1190844.0, ['See "Explanation of Responses"']]
[0, 0, 6, 'Acuta Capital Fund, LP', 2592743.0, ['10%']]
[0, 0.04, 10, 'Korenko Michael K', 173334.0, ['INTERIM CEO', 'Pres']]
[0, 0, 4, 'Poyhonen John', 313680.0, ['Dir']]
[0, 0.02, 14, 'Alta Fundamental Advisers LLC', 12930295.0, ['Dir']]
[0, 0, 5, "O'Neill Joseph C Jr", 177281.0, ['EVP', 'CFO']]
[0, 0, 8, 'Wickersham Grover T.', 491954.0, ['Dir']]
[0, 0, 4, 'Farrell Mark L', 109023.0, ['Dir']]
[0, 0, 7, 'Haley Marie', 20157.0, ['EVP', 'Chief Retail Officer']]
[0, 0, 5, 'Kola Ismail', 142302.0, ['Dir']]
[0, 0, 8, 'Murley Robert S', 947892.0, ['Dir']]
[0, -0.06, 4, 'Munch Robert J', 122831.0, ['Dir']]
[0, 0, 4, 'Owensby Jeffrey', 119080.4, ['Dir']]
[0, 0, 5, 'Van Kesteren Gerard', 144386.0, ['Dir']]
[0, 0, 12, 'Lloyd Karole', 1413534.0, ['Dir']]
[0, 0, 5, 'Trieu Vuong', 272605.0, ['COB', '10%']]
[0, 0, 5, "Vos Ellen O'Connor", 205213.0, ['Dir']]
[0, 0, 7, 'Brannock 

[-0.0006217616580310881, -0.04, 4, 'Libassi Joseph J', 129166.0, ['Chairman of the Board']]
[-0.0006255424023436445, -0.58, 8, 'Mosing D. Keith', 6411108.0, ['See Remarks', '10%']]
[-0.0006267029972752044, -0.92, 4, 'Parsow Alan S', 22530.0, ['Dir']]
[-0.0006271170754280426, -0.22, 5, 'Wobensmith John C', 324960.0, ['CFO', 'Secretary', 'Treasurer']]
[-0.0006289819653788201, -0.29, 41, 'Quercus Trust', 63547776.0, ['10%']]
[-0.0006312292358803986, -0.38, 4, 'Singer James  R', 12017237.0, ['10%']]
[-0.000631768953068592, -0.7, 7, 'Ustaszewski John', 790788.0, ['VP', 'CFO']]
[-0.00063188154209002, -0.55, 25, 'Freibaum Bernard', 31025357.0, ['CFO']]
[-0.0006320907617504052, -0.39, 8, 'Sarachek Russell M', 66471.0, ['Dir']]
[-0.0006352113801756891, -0.55, 4, 'Moyers Dwayne A', 52435.0, ['COB ', 'CEO']]
[-0.0006354786177012863, -0.43, 77, 'Brown J McCauley', 5717281.48, ['10%']]
[-0.0006361731437436982, -0.32, 5, 'Kinzel Richard L', 470610.0, ['COB', 'Pres', 'CEO']]
[-0.0006422018348623854, 

OrderedDict([('Dir',
              [0.64,
               0.46350758853288365,
               0.22733957219251338,
               0.10995121951219512,
               0.07893841525860221,
               0.06,
               0.057338633631619514,
               0.03604302477183833,
               0.02982090072348045,
               0.02180628566557413,
               0.01893617021276596,
               0.017454936064928938,
               0.01570520965692503,
               0.01518604550096695,
               0.012789050245408086,
               0.012580645161290323,
               0.01249986076996213,
               0.011501811868867954,
               0.011481232788711522,
               0.011259811266357668,
               0.010485436893203885,
               0.010332613818034245,
               0.0095049238143677,
               0.009487179487179488,
               0.009261363636363635,
               0.009241783336384571,
               0.009207262027641269,
               0.00889937

In [50]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np

boxPosRetData = list(positionStats.values())

fig = plt.figure(figsize =(900, 9))

# Creating axes instance
ax = fig.add_axes([0, 0, 1, 1])

titlesCounts = []
for pos in positionStats:
    titlesCounts.append(pos+' ('+str(len(positionStats[pos]))+')')

# x-axis labels
ax.set_xticklabels(titlesCounts)

plt.title('Returns By Title')
plt.xlabel('Position')
plt.ylabel('Return (Profit/Invested)')
plt.xticks(rotation=90)

#log scale y-axis to make comparison easier
ax.set_yscale('log')

# Creating plot
bp = ax.boxplot(boxPosRetData)

# show plot
plt.show()


/var/folders/cn/bl_17ghs1c36tj34j2pxt3gh0000gn/T/ipykernel_1327/3907242924.py:17: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(titlesCounts)


##### Conclusions and Points of Improvement (many)

Conclusions:
-look at positions that protect downside (top of their bottom quartile---iow the bottom of the box---is high) in addition to, of course, having a relatively high median and upside

-by this aforementioned standard, top positions to follow are: CFO, VP, GC (General Counsel), Marketing, VP Finance, CMO (Marketing repeated), VP Sales, Chief Commercial Officer



Improve:

-consider the time that traders held stock. What insiders are getting quick returns on their purchases? (or more importantly which individuals? this may be more of an individual-based measure than position-based). there is the chance that many buys/sells were over the course of a decade, in which case a 1x return doesn't look so good compared to a 1x over 2 years (this may explain cfo's and vp's having high returns---maybe they hold their shares for a long time as they stay with the company...I must always be sure to check my assumptions)
    
---update^: did this by dividing returns by the number of days held for a given share

-make returns measuring weighted by the volume of trades made, assuming that we care more about those with large trade volumes than those with small trades??? (then again this somewhat implies/assumes that people making smaller trades are less credible and more reckless with their money, which may or may not be true in reality...so maybe don't weight it)

In [61]:
import statistics
# Lets get the average of the median and the bottom quartile cutoff for each position (protect against the worst)
# Then we'll rank positions by that average
avgQ1Q2 = []

for pos in positionStats:
    avgQ1Q2.append([(np.quantile(positionStats[pos], .50) + np.quantile(positionStats[pos], .25))/2, pos, len(positionStats[pos])])
#     avgQ1Q2.append([(np.quantile(positionStats[pos], .50) + np.quantile(positionStats[pos], .25))/2, pos+' ('+str(len(positionStats[pos]))+')'])

# avgQ1Q2
# sorted(avgQ1Q2, key=lambda x: x[2], reverse=True)
list(filter(lambda x: x[2]>10, sorted(avgQ1Q2, key=lambda x: x[0], reverse=True)))



[[0.0009308465388451904, 'CTO', 11],
 [0.0008468097412584104, 'Sec.', 11],
 [0.0007632113893733033, 'VP Finance', 16],
 [0.0004994129774855193, 'Gen. Counsel', 11],
 [0.00040895856370522395, 'Chief Accounting Officer', 22],
 [0.0003740002287001928, 'GC', 78],
 [0.0002857131120444545, 'VP', 228],
 [0.00025333554113679275, 'Controller', 34],
 [0.00023613752682100339, 'CFO', 492],
 [0.0002238272707372384, 'See Remarks', 26],
 [0.00021576737113762555, 'CIO', 19],
 [0.00021336656398113002, 'SVP', 280],
 [0.0002091683607723137, 'Secretary', 80],
 [0.00019984272647244014, 'Exec COB', 16],
 [0.00019748948130768958, 'Pres', 837],
 [0.00017808226441815505, 'CEO', 997],
 [0.00015766388941664225, 'COO', 169],
 [0.00015351441702760184, 'EVP', 353],
 [0.00015284122666547103, 'CAO', 13],
 [0.00014456936048210043, 'Chief Credit Officer', 11],
 [0.00012398271726743587, 'COB', 310],
 [0.00012298124861912397, 'Vice COB', 23],
 [0.00010997308868986924, 'Dir', 2947],
 [6.752704223154808e-05, 'Treasurer', 7

# -----conclusion divider----

In [14]:
data.loc[data['Title'] == 'Marketing']

,Trade Date,Ticker,Insider Name,Company Name,Title,Trade Type,Price,Qty,Owned,deltOwned,Value


In [79]:
# print(data.loc[data['Title'].str.contains('CTO', regex=False, na=False)])

data.loc[data['Title'].str.contains('Sec.', regex=False, na=False)].loc[data['Trade Type'].str.contains('P - Purchase', regex=False, na=False)]


,Trade Date,Ticker,Insider Name,Company Name,Title,Trade Type,Price,Qty,Owned,deltOwned,Value
1638,1/8/2004,ANTP,Wraight Clark D,Phazar Corp,"VP, Sec.Treas.",P - Purchase,6.23,200,155965,0,1246
1874,1/9/2004,ABCB,Lewis Cindi H,Abc Bancorp,"Dir. of H, R, Corp. Sec.",P - Purchase,17.23,61,24239,0,1051
1913,1/9/2004,ABCB,Lewis Cindi H,Abc Bancorp,"Dir. of H, R, Corp. Sec.",P - Purchase,17.23,61,24239,0,1051
1933,1/9/2004,TSCO,Lewis David C,Tractor Supply Co /de/,"VP-Controller, Corp. Sec.",P - Purchase,27.89,94,2884,3,2621
6016,1/29/2004,ANTP,Wraight Clark D,Phazar Corp,"VP, Sec.Treas.",P - Purchase,8.27,200,156165,0,1654
...,...,...,...,...,...,...,...,...,...,...,...
1213218,11/1/2021,RGCO,Oliver Lawrence T.,Rgc Resources Inc,"VP, Interim CFO, Sec., Treas.",P - Purchase,22.26,9,4340,0,200
1216402,11/15/2021,UAN,Buhrig Melissa M,"Cvr Partners, LP","EVP, GC, Sec.",P - Purchase,77.25,2200,2200,0,169950.0
1219222,11/29/2021,EAT,Fuller Daniel S,"Brinker International, Inc","SVP, GC, Sec.",P - Purchase,35.42,1275,24457,5,45161
1219914,12/1/2021,RGCO,Oliver Lawrence T.,Rgc Resources Inc,"VP, Interim CFO, Sec., Treas.",P - Purchase,21.86,9,4349,0,200


In [72]:
data.loc[data['Insider Name'] == 'Huang Charles Chin']

,Trade Date,Ticker,Insider Name,Company Name,Title,Trade Type,Price,Qty,Owned,deltOwned,Value
240,1/2/2004,ANAD,Huang Charles Chin,Anadigics Inc,CTO,P - Purchase,0.11,335634,335634,0,36919.74
291,1/2/2004,ANAD,Huang Charles Chin,Anadigics Inc,CTO,P - Purchase,0.10,325634,325634,0,32563.4
10527,2/17/2004,ANAD,Huang Charles Chin,Anadigics Inc,CTO,S - Sale+OE,7.55,-10000,335634,-3,-75500
12242,2/20/2004,ANAD,Huang Charles Chin,Anadigics Inc,CTO,P - Purchase,0.21,56661,347495,19,11787
185978,2/21/2006,ANAD,Huang Charles Chin,Anadigics Inc,CTO,S - Sale,6.55,-3480,418729,-1,-22794
225844,8/7/2006,ANAD,Huang Charles Chin,Anadigics Inc,CTO,S - Sale,5.09,-5088,438248,-1,-25898
233438,9/5/2006,ANAD,Huang Charles Chin,Anadigics Inc,CTO,S - Sale+OE,8.36,-10000,886496,-1,-83600
247256,11/13/2006,ANAD,Huang Charles Chin,Anadigics Inc,CTO,S - Sale+OE,8.80,-15000,891496,-2,-132000
251016,11/27/2006,ANAD,Huang Charles Chin,Anadigics Inc,CTO,S - Sale+OE,9.86,-20000,896496,-2,-197200
279309,3/16/2007,ANAD,Huang Charles Chin,Anadigics Inc,CTO,S - Sale,12.60,-12419,479575,-3,-156479


In [73]:
# lets also find insiders (for insider name-specific conclusions) who posted top returns and traded MULTIPLE tickers

In [74]:
insiders['Huang Charles Chin']

{'Title(s)': ['CTO'],
 'Affiliated Company': 'Anadigics Inc',
 'Profit': 3792535.7199999997,
 'Amount Inv': 81270.14,
 'Return': 46.67,
 'Equity Dist': {'ANAD': 81270.14},
 'Trade Count': 3,
 'Shares Traded': 717929,
 'Shares Held': 717929,
 'Trade Data': [['1/2/2004',
   'ANAD',
   'Huang Charles Chin',
   'Anadigics Inc',
   'CTO',
   'P - Purchase',
   0.11,
   335634,
   335634,
   0,
   36919.74],
  ['1/2/2004',
   'ANAD',
   'Huang Charles Chin',
   'Anadigics Inc',
   'CTO',
   'P - Purchase',
   0.1,
   325634,
   325634,
   0,
   32563.4],
  ['2/20/2004',
   'ANAD',
   'Huang Charles Chin',
   'Anadigics Inc',
   'CTO',
   'P - Purchase',
   0.21,
   56661,
   347495,
   '19',
   '11787']],
 'Trades by Ticker': {'ANAD': [[1218, 0.11, '1/2/2004'],
   [325634, 0.1, '1/2/2004'],
   [56661, 0.21, '2/20/2004']]},
 'Avg Hold Stats': [0, 0, 0]}

In [78]:
# print(data.loc[data['Title'].str.contains('CTO', regex=False, na=False)])

data.loc[data['Title'].str.match('CTO', na=False)].loc[data['Trade Type'].str.contains('P - Purchase', regex=False, na=False)]


,Trade Date,Ticker,Insider Name,Company Name,Title,Trade Type,Price,Qty,Owned,deltOwned,Value
240,1/2/2004,ANAD,Huang Charles Chin,Anadigics Inc,CTO,P - Purchase,0.11,335634,335634,0,36919.74
291,1/2/2004,ANAD,Huang Charles Chin,Anadigics Inc,CTO,P - Purchase,0.10,325634,325634,0,32563.4
1086,1/6/2004,EAG,Cubley H Dean Dr,Eagle Broadband Inc,CTO,P - Purchase,1.30,8815,10055261,0,11460
4571,1/23/2004,FLWS,Micali Enzo J,1 800 Flowers Com Inc,CTO,P - Purchase,3.65,17160,0,-100,62634
7257,2/5/2004,EAG,Cubley H Dean Dr,Eagle Broadband Inc,CTO,P - Purchase,1.72,6649,1064965,1,11436
...,...,...,...,...,...,...,...,...,...,...,...
1220011,12/1/2021,ACIW,Puppala Ram Kumar,"Aci Worldwide, Inc.",CTO,P - Purchase,28.49,2000,18625,12,56980
1220351,12/1/2021,PRLB,Baker Arthur R. III,Proto Labs Inc,CTO,P - Purchase,49.21,10000,30613,49,492100
1220466,12/1/2021,ACIW,Puppala Ram Kumar,"Aci Worldwide, Inc.",CTO,P - Purchase,28.49,2000,18625,12,56980
1223126,12/15/2021,SURG,Mott John,"Surgepays, Inc.",CTO,P - Purchase,2.21,1000,8188,14,2210
